In [8]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split

In [21]:
folder_path=glob("../../data/*")
data1_img_path=glob(folder_path[0]+"/data/*.jpg")
data2_img_path=glob(folder_path[1]+"/data/*.jpg")
data3_img_path=glob(folder_path[2]+"/data/*.jpg")
data4_img_path=glob(folder_path[3]+"/data/*.jpg")

def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df



In [22]:
data1_img_path.extend(data2_img_path)
data1_img_path.extend(data3_img_path)
data1_img_path.extend(data4_img_path)

In [11]:
file_path=[]
with open(folder_path[0]+'/annotations.json') as f:
    annotations_json = json.load(f)
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data1_img_path if str(frame).zfill(5) in s] 
    file_path.append(file[0])
with open(folder_path[1]+'/annotations.json') as f:
    annotations_json = json.load(f)   
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data2_img_path if str(frame).zfill(5) in s] 
    file_path.append(file[0])

with open(folder_path[2]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data3_img_path if str(frame).zfill(5) in s] 
    file_path.append(file[0])
with open(folder_path[3]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data4_img_path if str(frame).zfill(5) in s] 
    file_path.append(file[0])
file_path=list(set(file_path))

In [12]:
standard_file=[]
TF_standard=[]
for i in range(len(file_path)):
    TF_standard.append(1)
x_train, x_test, y_train, y_test = train_test_split(np.array(file_path), np.array(TF_standard), test_size=0.2, random_state=42)
for i in range(len(x_train)):
    shutil.copy(x_train[i],'../../data/original_dataset/StandardFrame/Train/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/Train/'+str(i)+'.jpg')   
for i in range(len(x_test)):
    shutil.copy(x_test[i],'../../data/original_dataset/StandardFrame/Test/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/Test/'+str(i)+'.jpg') 

In [23]:
not_standard_file1=list(set(data1_img_path)-set(file_path))

for i in range(len(not_standard_file1)):
    
    TF_standard.append(0)
x_train, x_test, y_train, y_test = train_test_split(np.array(not_standard_file1), np.zeros(len(not_standard_file1)), test_size=0.2, random_state=42)
for i in range(len(x_train)):
    shutil.copy(not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Train/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/Train/'+str(i)+'.jpg')
for i in range(len(x_test)):
    shutil.copy(not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Test/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/Test/'+str(i)+'.jpg')

In [24]:
original_standard_frame = pd.DataFrame({'file_path':standard_file,
                             'standard_frame':TF_standard})


In [26]:
original_standard_frame.to_csv('../../data/original_dataset/data_set.csv', index = None) #csv파일로 생성